In [ ]:

# Explanation of code 1 & 2 (real time rates display)

#This code fetches real-time currency exchange rates using an API and displays the results in a structured format. 
#It imports the requests library for sending HTTP requests and the pandas library for data manipulation. 
#An access key from the apilayer API service is used for authentication. 
#The code defines the API endpoint URL and parameters, including the access key, target currencies (EUR, GBP, CAD, PLN), 
# base currency (USD), and response format (JSON).
# It sends a GET request to the API and checks for a successful response.
# If successful, the response is converted from JSON to a dictionary, the USD to EUR exchange rate is extracted, 
# and the data is organized into a pandas DataFrame for display.

In [23]:
# code 1
# Function & Technology 1 
# currency exchange rate API from website 2

import requests
import pandas as pd

# Define your new access key
access_key = 'b2292154bb69a61f8686faf5294c39db'

# Define the URL for the API request
url = 'http://data.fixer.io/api/latest'

# Define the parameters for the API request
params = {
    'access_key': access_key,
    'symbols': 'USD'
}

# Make the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    if data['success']:
        # Extract the EUR to USD exchange rate
        eur_to_usd_rate = data['rates']['USD']
        
        # Calculate the USD/EUR rate
        usd_to_eur_rate = 1 / eur_to_usd_rate
        
        # Create a DataFrame to display the USD/EUR rate
        df = pd.DataFrame([['USD/EUR', usd_to_eur_rate]], columns=['Currency Pair', 'Exchange Rate'])
        
        # Display the DataFrame
        print(df)
    else:
        print(f"Error: {data['error']['info']}")
else:
    print(f"Failed to retrieve data: {response.status_code}")





  Currency Pair  Exchange Rate
0       USD/EUR        0.93121


In [4]:
# code 2
# Function & Technology 1 
# currency exchange rate API from website 1
import requests
import pandas as pd

# Define your access key
access_key = '8763cdbfe55bbc20458719f6b086e96d'

# Define the URL for the API request
url = 'http://apilayer.net/api/live'

# Define the parameters for the API request
params = {
    'access_key': access_key,
    'currencies': 'EUR,GBP,CAD,PLN',
    'source': 'USD',
    'format': 1
}

# Make the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    if data['success']:
        # Extract the USD to EUR exchange rate
        usd_to_eur_rate = data['quotes']['USDEUR']
        
        # Create a DataFrame to display the USD/EUR rate
        df = pd.DataFrame([['USD/EUR', usd_to_eur_rate]], columns=['Currency Pair', 'Exchange Rate'])
        
        # Display the DataFrame
        print(df)
    else:
        print(f"Error: {data['error']['info']}")
else:
    print(f"Failed to retrieve data: {response.status_code}")


  Currency Pair  Exchange Rate
0       USD/EUR       0.931998


In [ ]:
# Explanation of code 3 & 4(Mortgage and currency exchange rate prediction)
# The  code predicts future mortgage rates and EUR/CNY exchange rates using historical economic indicators. 
#The process involves importing libraries (pandas, ARIMA, sklearn), loading datasets, and converting date columns. 
# An ARIMA-based forecasting function generates future values for economic indicators, which are organized into DataFrames.

# For both predictions, features and target variables are separated, missing values are imputed, and data is split into training and testing sets. 
# A linear regression model with feature scaling is trained and used to predict future rates. 
# The results, displayed in DataFrames, offer valuable insights into future mortgage and currency exchange rates 
# based on comprehensive economic data.

In [30]:
# code 3
# Function & Technology 2
# Future mortgage rate prediction

import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_excel("C:/Users/user/Desktop/Data_mortgage.xlsx", engine='openpyxl')

# Ensure DateTime column is in datetime format
data['DateTime'] = pd.to_datetime(data['DateTime'])

# Forecasting function
def forecast_feature(data, feature_name, order=(5,1,0), steps=12):
    series = data[['DateTime', feature_name]].set_index('DateTime')
    series = series.asfreq('M')
    arima_model = ARIMA(series, order=order)
    arima_fit = arima_model.fit()
    return arima_fit.forecast(steps=steps)

# Forecast all features
consumer_confidence_forecast = forecast_feature(data, 'Consumer_Confidence')
cpi_forecast = forecast_feature(data, 'CPI')
housing_index_forecast = forecast_feature(data, 'Housing_index')
inflation_rate_forecast = forecast_feature(data, 'inflation_rate')
unemplyment_forecast = forecast_feature(data, 'unemplyment')
gdpg_forecast = forecast_feature(data, 'GDPG')
gytm_forecast = forecast_feature(data, 'GYTM')
stockindex_forecast = forecast_feature(data, 'STOCKINDEX')
lendingrate_forecast = forecast_feature(data, 'LENDINGRATE')
marginal_lending_facility_forecast = forecast_feature(data, 'Marginal_lending_facility')
main_refinancing_operations_forecast = forecast_feature(data, 'Main_refinancing_operations')
deposit_facility_forecast = forecast_feature(data, 'Deposit_facility')

# Create DataFrame with all forecasted features
forecasted_features = pd.DataFrame({
    'Consumer_Confidence': consumer_confidence_forecast.values,
    'CPI': cpi_forecast.values,
    'Housing_index': housing_index_forecast.values,
    'inflation_rate': inflation_rate_forecast.values,
    'unemplyment': unemplyment_forecast.values,
    'GDPG': gdpg_forecast.values,
    'GYTM': gytm_forecast.values,
    'STOCKINDEX': stockindex_forecast.values,
    'LENDINGRATE': lendingrate_forecast.values,
    'Marginal_lending_facility': marginal_lending_facility_forecast.values,
    'Main_refinancing_operations': main_refinancing_operations_forecast.values,
    'Deposit_facility': deposit_facility_forecast.values
}, index=cpi_forecast.index)

# Ensure the feature order is consistent
feature_columns = [
    'Consumer_Confidence', 'CPI', 'Housing_index', 'inflation_rate', 
    'unemplyment', 'GDPG', 'GYTM', 'STOCKINDEX', 'LENDINGRATE', 
    'Marginal_lending_facility', 'Main_refinancing_operations', 'Deposit_facility'
]

forecasted_features = forecasted_features[feature_columns]

# Split features and target variable
X = data[feature_columns]
y = data['Mortgage_rate']

# Handle missing values by imputing with the mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
forecasted_features_imputed = imputer.transform(forecasted_features)

# Split training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model with scaling
linear_regression_pipeline = make_pipeline(StandardScaler(), LinearRegression())
linear_regression_pipeline.fit(X_train, y_train)

# Scale forecasted features
forecasted_features_scaled = linear_regression_pipeline.named_steps['standardscaler'].transform(forecasted_features_imputed)

# Predict Mortgage Rate
future_mortgage_rate_predictions = linear_regression_pipeline.named_steps['linearregression'].predict(forecasted_features_scaled)

# Create a DataFrame to display the results
results = pd.DataFrame({
    'Year-Month': forecasted_features.index.strftime('%Y-%m'),
    'Estimated Mortgage Rate': future_mortgage_rate_predictions
})

# Output the results
print(results)


   Year-Month  Estimated Mortgage Rate
0     2024-06                 3.791551
1     2024-07                 3.763212
2     2024-08                 3.749859
3     2024-09                 3.751660
4     2024-10                 3.735493
5     2024-11                 3.726175
6     2024-12                 3.721581
7     2025-01                 3.715161
8     2025-02                 3.712926
9     2025-03                 3.708840
10    2025-04                 3.705287
11    2025-05                 3.702071


In [1]:
# code 4
# Function & Technology 2
# Future currency exchange rate prediction 

import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_excel("C:/Users/user/Desktop/Data_FX.xlsx", engine='openpyxl')

# Ensure DateTime column is in datetime format
data['DateTime'] = pd.to_datetime(data['DateTime'])

# Forecasting function
def forecast_feature(data, feature_name, order=(5,1,0), steps=12):
    series = data[['DateTime', feature_name]].set_index('DateTime')
    series = series.asfreq('M')
    arima_model = ARIMA(series, order=order)
    arima_fit = arima_model.fit()
    return arima_fit.forecast(steps=steps)

# Forecast all features
cpi_forecast = forecast_feature(data, 'CPI')
unemplyment_forecast = forecast_feature(data, 'unemplyment')
balance_of_trade_forecast = forecast_feature(data, 'Balance_of_Trade')
inflation_rate_forecast = forecast_feature(data, 'inflation_rate')
consumer_confidence_forecast = forecast_feature(data, 'Consumer_Confidence')
housing_index_forecast = forecast_feature(data, 'Housing_index')
capital_flows_forecast = forecast_feature(data, 'Capital_Flows')
fdi_forecast = forecast_feature(data, 'FDI')
gdpg_forecast = forecast_feature(data, 'GDPG')
gytm_forecast = forecast_feature(data, 'GYTM')
stockindex_forecast = forecast_feature(data, 'STOCKINDEX')
eurusd_forecast = forecast_feature(data, 'EURUSD')
lendingrate_forecast = forecast_feature(data, 'LENDINGRATE')
marginal_lending_facility_forecast = forecast_feature(data, 'Marginal_lending1_facility')
deposit_facility_forecast = forecast_feature(data, 'Deposit_facility')
foreign_exchange_reserves_forecast = forecast_feature(data, 'Foreign_Exchange_Reserves')

# Create DataFrame with all forecasted features
forecasted_features = pd.DataFrame({
    'CPI': cpi_forecast.values,
    'unemplyment': unemplyment_forecast.values,
    'Balance_of_Trade': balance_of_trade_forecast.values,
    'inflation_rate': inflation_rate_forecast.values,
    'Consumer_Confidence': consumer_confidence_forecast.values,
    'Housing_index': housing_index_forecast.values,
    'Capital_Flows': capital_flows_forecast.values,
    'FDI': fdi_forecast.values,
    'GDPG': gdpg_forecast.values,
    'GYTM': gytm_forecast.values,
    'STOCKINDEX': stockindex_forecast.values,
    'EURUSD': eurusd_forecast.values,
    'LENDINGRATE': lendingrate_forecast.values,
    'Marginal_lending1_facility': marginal_lending_facility_forecast.values,
    'Deposit_facility': deposit_facility_forecast.values,
    'Foreign_Exchange_Reserves': foreign_exchange_reserves_forecast.values
}, index=cpi_forecast.index)

# Ensure the feature order is consistent
feature_columns = [
    'Balance_of_Trade', 'Consumer_Confidence', 'CPI', 'Housing_index',
    'inflation_rate', 'unemplyment', 'Capital_Flows', 'FDI', 'GDPG', 'GYTM',
    'STOCKINDEX', 'EURUSD', 'LENDINGRATE', 'Marginal_lending1_facility',
    'Deposit_facility', 'Foreign_Exchange_Reserves'
]

forecasted_features = forecasted_features[feature_columns]

# Split features and target variable
X = data[feature_columns]
y = data['EURCNY']

# Handle missing values by imputing with the mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
forecasted_features_imputed = imputer.transform(forecasted_features)

# Split training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model with scaling
linear_regression_pipeline = make_pipeline(StandardScaler(), LinearRegression())
linear_regression_pipeline.fit(X_train, y_train)

# Scale forecasted features
forecasted_features_scaled = linear_regression_pipeline.named_steps['standardscaler'].transform(forecasted_features_imputed)

# Predict EURCNY
future_eurcny_predictions = linear_regression_pipeline.named_steps['linearregression'].predict(forecasted_features_scaled)

# Create a DataFrame to display the results
results = pd.DataFrame({
    'Year-Month': forecasted_features.index.strftime('%Y-%m'),
    'Estimated EURCNY': future_eurcny_predictions
})

# Output the results
print(results)

   Year-Month  Estimated EURCNY
0     2024-06          7.852322
1     2024-07          7.867048
2     2024-08          7.869139
3     2024-09          7.903796
4     2024-10          7.902884
5     2024-11          7.915911
6     2024-12          7.924863
7     2025-01          7.933650
8     2025-02          7.946828
9     2025-03          7.954492
10    2025-04          7.963170
11    2025-05          7.970341


In [2]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# explanation of code 5 (mortgage calculator)
# This code implements an interactive mortgage calculator using ipywidgets and IPython.display. 
# It defines a function, calculate_mortgage, to compute the maximum mortgage amount, gross monthly expenses, 
# and total own funds based on user inputs such as annual income, interest rate, purchase price, energy label, and mortgage type. 
# Various interactive widgets are created for these inputs, along with a button to trigger the calculation. 
# When the button is clicked, the function calculates the mortgage details and displays the results in HTML format 
# within the notebook interface, providing users with an easy-to-use tool for mortgage planning.

In [3]:
# code 5
# Function & Technology 3
# Mortgage calculator 

import ipywidgets as widgets
from IPython.display import display, HTML

# Function to calculate the mortgage loan amount
def calculate_mortgage(annual_income, interest_rate, purchase_price, energy_label, mortgage_type):
    energy_label_multiplier = {
        'A': 5.0,
        'B': 4.9,
        'C': 4.8,
        'D': 4.7,
        'E': 4.6,
        'F': 4.5,
        'G': 4.4
    }
    
    multiplier = energy_label_multiplier.get(energy_label, 4.5)
    max_mortgage = annual_income * multiplier
    
    if mortgage_type == 'Annuity':
        monthly_interest_rate = interest_rate / 100 / 12
        number_of_payments = 30 * 12
        gross_monthly_expenses = (max_mortgage * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -number_of_payments)
    else:  # Linear mortgage
        monthly_principal_repayment = max_mortgage / (30 * 12)
        initial_interest_payment = max_mortgage * (interest_rate / 100 / 12)
        initial_monthly_payment = monthly_principal_repayment + initial_interest_payment
        gross_monthly_expenses = initial_monthly_payment
    
    own_funds = purchase_price - max_mortgage
    min_own_funds = own_funds
    taxes = 6000
    broker_fee = 4500
    appraisal_fee = 650
    mortgage_advice_fee = 1995
    notary_fee = 1500
    total_own_funds = min_own_funds + taxes + broker_fee + appraisal_fee + mortgage_advice_fee + notary_fee
    
    return max_mortgage, gross_monthly_expenses, total_own_funds, min_own_funds, taxes, broker_fee, appraisal_fee, mortgage_advice_fee, notary_fee

# Create input widgets
annual_income_input = widgets.FloatText(value=60000, description='Annual Income (€):')
interest_rate_input = widgets.FloatText(value=4, description='Interest Rate (%):')
purchase_price_input = widgets.FloatText(value=300000, description='Purchase Price (€):')
energy_label_input = widgets.Dropdown(options=['A', 'B', 'C', 'D', 'E', 'F', 'G'], value='B', description='Energy Label:')
mortgage_type_input = widgets.Dropdown(options=['Annuity', 'Linear'], value='Annuity', description='Mortgage Type:')
calculate_button = widgets.Button(description='Calculate Mortgage')
output = widgets.Output()

# Define the button click event
def on_calculate_button_clicked(b):
    with output:
        output.clear_output()
        annual_income = annual_income_input.value
        interest_rate = interest_rate_input.value
        purchase_price = purchase_price_input.value
        energy_label = energy_label_input.value
        mortgage_type = mortgage_type_input.value
        results = calculate_mortgage(annual_income, interest_rate, purchase_price, energy_label, mortgage_type)
        max_mortgage, gross_monthly_expenses, total_own_funds, min_own_funds, taxes, broker_fee, appraisal_fee, mortgage_advice_fee, notary_fee = results
        display(HTML(f"""
        <h2>Mortgage Calculation Results</h2>
        <p><strong>Maximum Mortgage Amount:</strong> €{max_mortgage:,.2f}</p>
        <p><strong>Gross Monthly Expenses:</strong> €{gross_monthly_expenses:,.2f}</p>
        <p><strong>Total Own Funds:</strong> €{total_own_funds:,.2f}</p>
        <ul>
            <li>Minimum Own Funds: €{min_own_funds:,.2f}</li>
            <li>Taxes: €{taxes:,.2f}</li>
            <li>Broker Fee: €{broker_fee:,.2f}</li>
            <li>Appraisal Fee: €{appraisal_fee:,.2f}</li>
            <li>Mortgage Advice Fee: €{mortgage_advice_fee:,.2f}</li>
            <li>Notary Fee: €{notary_fee:,.2f}</li>
        </ul>
        """))

# Bind the button click event
calculate_button.on_click(on_calculate_button_clicked)

# Display the widgets
display(annual_income_input, interest_rate_input, purchase_price_input, energy_label_input, mortgage_type_input, calculate_button, output)


FloatText(value=60000.0, description='Annual Income (€):')

FloatText(value=4.0, description='Interest Rate (%):')

FloatText(value=300000.0, description='Purchase Price (€):')

Dropdown(description='Energy Label:', index=1, options=('A', 'B', 'C', 'D', 'E', 'F', 'G'), value='B')

Dropdown(description='Mortgage Type:', options=('Annuity', 'Linear'), value='Annuity')

Button(description='Calculate Mortgage', style=ButtonStyle())

Output()